In [ ]:
Estudiante : Winny Lesly Quispe Gutierrez
Chatbot Huellitas (tienda de productos para mascotas)
link archivo json https://github.com/wwwinny/Huellitas-ChatBot.git

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!pip install tflearn


In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

#Libararies needed for TensorFlow processing
import tensorflow as tf
import numpy as np
import tflearn
import random
import json
#cargamos los datos json
from google.colab import files
files.upload()


Saving intentos.json to intentos (1).json


{'intentos.json': b'{\r\n    "intents": [{\r\n            "tag": "saludo",\r\n            "patterns": ["Hola", "\xc2\xbfC\xc3\xb3mo est\xc3\xa1s", "\xc2\xbfHay alguien ah\xc3\xad?", "Buen d\xc3\xada", "Buenas tardes", "Buenos d\xc3\xadas", "Buenas noches"],\r\n            "responses": ["Hola, gracias por visitar nuestra tienda de productos para mascotas Huellitas", "Es bueno verte , bienvenido a tienda de productos para mascotas Huellitas", "Hola, \xc2\xbfen qu\xc3\xa9 puedo ayudar?", "Hola, \xc2\xbf Que desea hacer?"],\r\n            "context_set": ""\r\n        },\r\n        {\r\n            "tag": "adios",\r\n            "patterns": ["Adios", "Chau", "Hasta luego", "Hasta otra ", "Hasta la vista"],\r\n            "responses": ["Nos vemos, gracias por visitar nuestra tienda Huellitas", "Ten un buen dia", "Adios, vuelva pronto", "La Tienda Huellitas le agradece su visita ", "Adios, espero verlo denuevo"],\r\n            "context_set": ""\r\n        },\r\n        {\r\n            "tag"

In [ ]:
with open('intentos.json') as json_data:
    intents = json.load(json_data)

In [ ]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hola',
    '¿Cómo estás',
    '¿Hay alguien ahí?',
    'Buen día',
    'Buenas tardes',
    'Buenos días',
    'Buenas noches'],
   'responses': ['Hola, gracias por visitar nuestra tienda de productos para mascotas Huellitas',
    'Es bueno verte , bienvenido a tienda de productos para mascotas Huellitas',
    'Hola, ¿en qué puedo ayudar?',
    'Hola, ¿ Que desea hacer?'],
   'tag': 'saludo'},
  {'context_set': '',
   'patterns': ['Adios',
    'Chau',
    'Hasta luego',
    'Hasta otra ',
    'Hasta la vista'],
   'responses': ['Nos vemos, gracias por visitar nuestra tienda Huellitas',
    'Ten un buen dia',
    'Adios, vuelva pronto',
    'La Tienda Huellitas le agradece su visita ',
    'Adios, espero verlo denuevo'],
   'tag': 'adios'},
  {'context_set': '',
   'patterns': ['Gracias',
    'Eso me ayudo',
    'Gracias por la ayuda',
    'Gracias por la consulta'],
   'responses': ['Feliz de poder ser de ayuda',
    'Es un placer',
   

In [ ]:
palabras = []
etiquetas = []
documentos = []
ignore = ['?']

#recorre cada oracion 
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w=nltk.word_tokenize(pattern)
        palabras.extend(w)
        documentos.append((w, intent['tag']))
        if intent['tag'] not in etiquetas:
            etiquetas.append(intent['tag'])

In [ ]:
#elimina dupplicados y reduce palabras
palabras=[stemmer.stem(w.lower()) for w in palabras if w not in ignore]
palabras=sorted(list(set(palabras)))

#remove duplicate classes
etiquetas=sorted(list(set(etiquetas)))

print (len(documentos), "documentos")
print (len(etiquetas), "etiquetas", etiquetas)
print (len(palabras), "palabras derivadas únicas", palabras)

37 documentos
9 etiquetas ['abierto', 'adios', 'cortes', 'gracias', 'horario', 'pagos', 'productos', 'saludo', 'vacunas']
86 palabras derivadas únicas ['a', 'abierto', 'abr', 'adio', 'ahí', 'alguy', 'at', 'atendiendo', 'ayud', 'ayudo', 'buen', 'buena', 'bueno', 'chau', 'con', 'consult', 'cort', 'crédito', 'cuest', 'de', 'dia', 'día', 'efectivo', 'el', 'en', 'es', 'eso', 'est', 'está', 'gato', 'gracia', 'habr', 'hast', 'hay', 'hol', 'hor', 'hora', 'horario', 'hoy', 'la', 'luego', 'mastercard', 'mañan', 'me', 'noch', 'o', 'otr', 'pag', 'par', 'pelo', 'perro', 'por', 'producto', 'puedo', 'que', 'qué', 'solo', 'son', 'sus', 'tard', 'tarjeta', 'tien', 'tiend', 'un', 'vacuna', 'vacunac', 'vend', 'vent', 'vist', 'yap', '¿a', '¿acepta', '¿aceptan', '¿como', '¿cual', '¿cuáles', '¿cuándo', '¿cuánto', '¿cómo', '¿es', '¿está', '¿habra', '¿hay', '¿puedo', '¿que', '¿realiza']


In [ ]:
# entrenamiento
entrenamiento=[]
salidas=[]
#crea una matriz vacía para la salida
output_empty=[0]*len(etiquetas)

for doc in documents:
  bag=[]
  pattern_words=doc[0]
  pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
  for w in palabras:
    bag.append(1) if w in pattern_words else bag.append(0)
  
  output_row=list(output_empty)
  output_row[etiquetas.index(doc[1])]=1
  entrenamiento.append([bag,output_row])
random.shuffle(entrenamiento)
entrenamiento=np.array(entrenamiento)
  #entrenamiento de listas
train_x =list(entrenamiento[:,0])
train_y =list(entrenamiento[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

net=tflearn.input_data(shape=[None, len(train_x[0])])
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,len(train_y[0]), activation='softmax')
net=tflearn.regression(net)

model=tflearn.DNN(net, tensorboard_dir='tflearn_logs')

#iniciando entrenamiento
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 4999  | total loss: 0.67901 | time: 0.012s
| Adam | epoch: 1000 | loss: 0.67901 - acc: 0.9671 -- iter: 32/37
Training Step: 5000  | total loss: 0.55222 | time: 0.015s
| Adam | epoch: 1000 | loss: 0.55222 - acc: 0.9671 -- iter: 37/37
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
import pickle
pickle.dump({'palabras':palabras, 'etiquetas':etiquetas, 'train_x':train_x, 'train_y':train_y}, open("training_data","wb"))

In [ ]:
# restaurando todas las estructuras de datos
data = pickle.load(open("training_data","rb"))

palabras=data['palabras']
etiquetas =data['etiquetas']
train_x =data['train_x']
train_y =data['train_y']

In [ ]:
with open('intentos.json') as json_data:
    intentos = json.load(json_data)

In [ ]:
#cargar el modelo guardado
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [ ]:
def clean_up_sentence(sentence):
  #tokenizando el toen
  sentence_words=nltk.word_tokenize(sentence)
  #derivando cada palabra
  sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

def bow(sentence, words, show_details=False):
  #tokenizando la oracion
  sentence_words=clean_up_sentence(sentence)
  #genernado grupos de palabras
  bag=[0] * len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w ==s:
        bag[i] = 1
        
        if show_details:
          print("encontrar grupo: %s" % w)
  return (np.array(bag))

In [ ]:
ERROR_THRESHOLD = 0.0
def classify(sentence):
  results = model.predict([bow(sentence, palabras)])[0]
  
  results = [[i,r] for i, r in enumerate(results) if r> ERROR_THRESHOLD]
  
  results.sort(key=lambda x: x[1], reverse=True)
  return_list=[]
  
  for r in results:
    return_list.append((etiquetas[r[0]], r[1]))
  
  return return_list

def responder(sentence, userID='123', show_details=False):
  results=classify(sentence)
  
  if results:
    while results:
      for i in intents['intents']:
        if i['tag'] == results[0][0]:
          return print(random.choice(i['responses']))
      results.pop(0)
  

In [ ]:
classify('¿A qué horas está abierto?')

[('horario', 0.9995184),
 ('saludo', 0.0003369365),
 ('pagos', 0.00014471405),
 ('abierto', 1.4988301e-15),
 ('gracias', 8.45524e-17),
 ('cortes', 1.1008208e-24),
 ('adios', 2.0109829e-28),
 ('vacunas', 5.0517496e-30),
 ('productos', 6.5769644e-32)]

In [ ]:
responder('hola')

Hola, gracias por visitar nuestra tienda de productos para mascotas Huellitas


In [ ]:
responder('"¿Que productos vende?"')

Huellitas vende : Comida de perro , gato , hamsters y canarios ; tambien tenemos ropa para mascotas , productos de higiene para mascotas, juguetes, camas y demas accesorios 


In [ ]:
responder('¿Habra atencion mañana?')

Si, Huellitas habre todos los dias con excepcion de feriados de 8am a 6pm


In [ ]:
responder('gracias')

De nada


In [ ]:
responder('hasta otra')

Adios, vuelva pronto
